In [1]:
import sys
import polars as pl
import plotly.express as px
sys.path.insert(0, '..')
from fs_thesis import sql, show
from sklearn.metrics import classification_report, confusion_matrix
import plotly.figure_factory as ff
import numpy as np
import pandas as pd

# 1. Data Pipeline Overview
This notebook uses the centralized data pipeline defined in `fs_thesis.data_loader`. Below is a documentation of the logic encapsulated in `load_final_data()`.

### A. Patient Demographics (Baseline)
We extract the following baseline features from the first admission (`t0_time`):
- **Identifier**: `subject_id`
- **Demographics**: `gender`, `anchor_age`, `race`, `marital_status`, `language`, `insurance`
- **Context**: `admission_type`
- **BMI**: Median BMI from `hosp.omr` (Left Join, missing values are handled by TabPFN)

### B. Event Definition (Target)
The event is defined as the **first occurrence** of a specific ICD diagnosis (e.g., Heart Failure `I50%`).
- **Event Time**: Timestamp of the diagnosis.
- **Censoring**: If no event occurs, the patient is censored at the date of death (`dod`) or end of follow-up.

### C. Target Calculation Logic
The target variable is derived based on the time-to-event (`duration`):
1. **Calculate Duration**:
   - `t_event` = Days from baseline to diagnosis.
   - `t_death` = Days from baseline to death.
   - Priority: Event Time > Death Time > Fallback (2000 days).
   - Negative durations are clipped to 0.

2. **Define Classes (`target`)**:
   - **Class 0 (Early Event)**: Event occurs $\le$ 365 days.
   - **Class 1 (Late Event)**: Event occurs $>$ 365 days.
   - **Class 2 (Censored/Control)**: No event observed (censored or healthy).

In [2]:
from fs_thesis.data_loader import load_final_data
df_final = load_final_data()

In [3]:
# Prüfen wie viele Missings wir haben
print("Missing BMI from Join, before feature engineering: TabPFN will handle these missing values, but it's good to know how many we have.")
print(f"Missing BMI: {df_final['bmi'].null_count()} of {len(df_final)}")

Missing BMI from Join, before feature engineering: TabPFN will handle these missing values, but it's good to know how many we have.
Missing BMI: 98306 of 223452


# 2. Preprocessing
## Splitting

In [4]:
from fs_thesis.preprocessing import preprocess_data, balance_data, get_X_y
df_train, df_val, df_test = preprocess_data(df_final)

Shapes -> Train: (143008, 17), Val: (35753, 17), Test: (44691, 17)


## Sampling (Balancing)

In [5]:
# Balance (only for train data!)
df_balanced = balance_data(df_train, n_samples=3000)

# split Features & Target (all Sets!)
X_train, y_train = get_X_y(df_balanced)
X_val, y_val = get_X_y(df_val)
X_test, y_test = get_X_y(df_test)

# Jetzt passt auch der fucking Print
print(f"Train (balanced): {len(y_train)} | Val (real): {len(y_val)} | Test (real): {len(y_test)}")

Train (balanced): 9000 | Val (real): 35753 | Test (real): 44691


# 3. Training
## Classifier

In [9]:
from tabpfn import TabPFNClassifier
classifier = TabPFNClassifier(device='mps') # Zurück auf CPU, für schnellere Vorhersagen bei kleinen N

## Fit

In [10]:
print("Start fitting...")
classifier.fit(X_train, y_train)
print("Training done!")

Start fitting...
Training done!


## Predict

In [ ]:
# 3. Vorhersage (Validierung)
# Achtung: TabPFN ist bei Prediction auf vielen Daten (Validation Set = groß) langsam!
# Wir nehmen erstmal 1000 Samples für den schnellen Check.
X_val_sample = X_val.iloc[:1000]
y_val_sample = y_val[:1000]

# Sample run (faster)
sample = True
# For importance analysis
skip = True 

print("Start predict...")
y_val_pred = classifier.predict(X_val_sample) if sample else classifier.predict(X_val)
print("Done!")

Start predict...
Done!


# 4. Validation (val_set for optimizing)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

if sample:
    print(f"Accuracy: {accuracy_score(y_val_sample, y_val_pred):.2%}")
    print("\nClassification Report:")
    print(classification_report(y_val_sample, y_val_pred, 
                                target_names=['early (<1J)', 'late (>1J)', 'healthy']))
else:
    print(f"Accuracy: {accuracy_score(y_val, y_val_pred):.2%}")
    print("\nClassification Report:")
    print(classification_report(y_val, y_val_pred, 
                                target_names=['early (<1J)', 'late (>1J)', 'healthy']))

Accuracy: 57.10%

Classification Report:
              precision    recall  f1-score   support

 early (<1J)       0.12      0.75      0.21        40
  late (>1J)       0.12      0.81      0.21        36
     healthy       0.99      0.55      0.71       924

    accuracy                           0.57      1000
   macro avg       0.41      0.70      0.38      1000
weighted avg       0.92      0.57      0.67      1000



# 4.1 Visualization Data

In [ ]:
# Check: Zusammenhang zwischen Alter und Versicherung (Medicare)
# Wir schauen, wie hoch das Durchschnittsalter pro Versicherungsgruppe ist.
import plotly.express as px

if sample: 
    X_val = X_val_sample
    y_val = y_val_sample
else: 
    X_val = X_val
    y_val = y_val

df_analyze = X_val.copy() # Kopie für Analyse
if hasattr(df_analyze, "to_pandas"):
    df_analyze = df_analyze.to_pandas()

# Boxplot zeigt klar: Medicare-Patienten sind fast alle 65+
fig = px.box(df_analyze, x="insurance", y="anchor_age", 
             title="Warum Alter unwichtig wirkt: Der 'Medicare'-Effekt",
             points="all", 
             color="insurance")
fig.show()

In [ ]:
# 1. Daten vorbereiten (Scores berechnen)
y_proba = classifier.predict_proba(X_val)
# Wahrscheinlichkeit für Klasse 0 (Early Event)
risk_score = y_proba[:, 0]

df_analyze['risk_score'] = risk_score
df_analyze['true_label'] = y_val

### BMI Analyse

In [ ]:
bins = [0, 18.5, 25, 30, 100]
labels = ['Untergewicht (<18.5)', 'Normal (18.5-25)', 'Übergewicht (25-30)', 'Adipositas (>30)']
df_analyze['bmi_group'] = pd.cut(df_analyze['bmi'], bins=bins, labels=labels)

df_bmi_mean = df_analyze.groupby('bmi_group', observed=True)['risk_score'].mean().reset_index()

fig1 = px.bar(
    df_bmi_mean, 
    x='bmi_group', 
    y='risk_score',
    text_auto='.1%', # Zeigt %-Wert direkt auf dem Balken
    title='Avg risk by BMI-Group',
    labels={'risk_score': 'Risk Score', 'bmi_group': 'BMI Group'},
    color='risk_score',
    color_continuous_scale='Reds',
    template="plotly_white"
)
fig1.update_layout(yaxis_tickformat='.0%') # Y-Achse als Prozent formatieren
fig1.show()

### Age Analyse

In [ ]:
df_analyze['age_group'] = pd.cut(
    df_analyze['anchor_age'], 
    bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 120],  # 0-20, 20-30, ..., 90-120
    labels=['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
)
df_analyze['age_group'] = df_analyze['age_group'].astype(str)

df_age_mean = df_analyze.groupby('age_group')['risk_score'].mean().reset_index()

fig2 = px.bar(
    df_age_mean, 
    x='age_group', 
    y='risk_score',
    text_auto='.1%',
    title='Risk by Age Group',
    labels={'risk_score': 'Risk Score', 'age_group': 'Age Group'},
    color='risk_score',
    color_continuous_scale='Reds',
    template="plotly_white"
)
fig2.update_layout(yaxis_tickformat='.0%')
fig2.show()

### Social - Economy

In [ ]:
df_ins_mean = df_analyze.groupby(['insurance', 'gender'])['risk_score'].mean().reset_index()

fig3 = px.bar(
    df_ins_mean, 
    x='insurance', 
    y='risk_score', 
    color='gender', 
    barmode='group',
    text_auto='.1%',
    title='Risk by Gender and Insurance',
    labels={'risk_score': 'Risk Score', 'insurance': 'Insurance'},
    template="plotly_white"
)
fig3.update_layout(yaxis_tickformat='.0%')
fig3.show()

# 5. Visualisation Prediction

In [ ]:


if sample: 
    X_val = X_val_sample
    y_val = y_val_sample
else: 
    X_val = X_val
    y_val = y_val
# 1. Normalisierung der Confusion Matrix (Zeilenweise)
# Wie viel % der tatsächlichen Klasse wurden wie vorhergesagt?
# WICHTIG: Wir vergleichen hier mit y_val_sample, da wir nur darauf vorhergesagt haben!

cm = confusion_matrix(y_val, y_val_pred)
cm_perc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

labels = ['early (<1J)', 'late (1-3J)', 'healthy']

# 2. Text für die Boxen (Absolute Zahl + Prozent)
annot_text = [
    [f"<b>{val}</b><br>({perc:.1%})" for val, perc in zip(row_val, row_perc)]
    for row_val, row_perc in zip(cm, cm_perc)
]

# 3. Interaktive Heatmap
fig = ff.create_annotated_heatmap(
    cm_perc, 
    x=labels, 
    y=labels, 
    annotation_text=annot_text, 
    colorscale='Reds' # 'Reds' hebt die Treffer besser hervor
)
if sample:
    fig.update_layout(
        title='Validation Check: Confusion Matrix (1000 Samples)',
        xaxis_title="Vorhersage des Modells",
        yaxis_title="Tatsächlicher Verlauf (MIMIC-Daten)",
        template="plotly_white",
        height=600
    )
else:
    fig.update_layout(
        title='Validation Check: Confusion Matrix (3000 Samples)',
        xaxis_title="Vorhersage des Modells",
        yaxis_title="Tatsächlicher Verlauf (MIMIC-Daten)",
        template="plotly_white",
        height=600
    )

fig.show()

In [ ]:
import plotly.graph_objects as go

# Daten aus deiner Confusion Matrix extrahieren
# Reihenfolge: [Früh, Spät, Gesund]

if sample: 
    X_val = X_val_sample
    y_val = y_val_sample
else: 
    X_val = X_val
    y_val = y_val

cm = confusion_matrix(y_val, y_val_pred)

# Labels für die Knoten
label_list = [
    "real: early", "real: late", "real: healthy", # Quellen (Links)
    "predicted: early", "predicted: late", "predicted: healthy" # Ziele (Rechts)
]

# Definition der Flüsse (Sankey-Struktur)
source = [0, 0, 0, 1, 1, 1, 2, 2, 2] # Index der Quellen
target = [3, 4, 5, 3, 4, 5, 3, 4, 5] # Index der Ziele
value = cm.flatten() # Die Zahlen aus deiner Matrix

# Farben definieren (Grün für korrekt, Rot für Fehler)
color_link = [
    'rgba(255, 90, 90, 0.4)', 'rgba(255, 90, 90, 0.2)', 'rgba(255, 90, 90, 0.1)', # Von Früh
    'rgba(255, 127, 14, 0.2)', 'rgba(255, 127, 14, 0.4)', 'rgba(255, 127, 14, 0.1)', # Von Spät
    'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.4)'    # Von Gesund
]

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15, thickness = 20, line = dict(color = "black", width = 0.5),
      label = label_list, color = "grey"
    ),
    link = dict(
      source = source, target = target, value = value, color = color_link
  ))])

fig.update_layout(title_text="Patient-Flow: Reality vs. Predicted", font_size=12)
fig.show()

In [ ]:
from sklearn.inspection import permutation_importance
import pandas as pd
import plotly.express as px

if skip:
    print("skipped")
else:
    print("Berechne Feature Importance (kann ca. 1-2 Min. dauern oder 20)...")

    if sample: 
        X_val = X_val_sample
        y_val = y_val_sample
    else:    
        X_val = X_val
        y_val = y_val

    result = permutation_importance(
        classifier, X_val, y_val, n_repeats=10, random_state=42, n_jobs=1
    )

    # Ergebnisse in ein DataFrame gießen
    importance_df = pd.DataFrame({
        'Feature': ['gender', 'anchor_age', 'insurance', 'language', 'marital_status', 'race', 'admission_type', 'bmi'],
        'Importance': result.importances_mean,
        'Std_Dev': result.importances_std
    }).sort_values(by='Importance', ascending=True) # Aufsteigend für horizontalen Plot

    # Plotly Bar Chart
    fig = px.bar(
        importance_df, 
        x='Importance', 
        y='Feature', 
        orientation='h',
        title='Which social facts are important?',
        labels={'Importance': '(Mean Decrease Accuracy)'}, # Wissenschaftlicher Label
        error_x='Std_Dev', # Zeigt die Variabilität der Wichtigkeit
        template="plotly_white",
        color='Importance',
        color_continuous_scale='Reds'
    )

    # NEU: Prozent-Formatierung für bessere Lesbarkeit
    fig.update_layout(height=500, xaxis_tickformat='.1%')
    fig.show()

skipped


# Vorschläge für Loop

# Vorschlag 1

In [ ]:
# --- 1. SETUP FUNKTIONEN ---

def get_base_data():
    """Lädt Daten und macht den initialen Split (Train/Val/Test)."""
    # Das entspricht deinen Zellen am Anfang
    df_final = load_final_data()
    df_train, df_val, df_test = preprocess_data(df_final)
    return df_train, df_val, df_test

def train_iteration(df_train, df_val, seed):
    """
    Führt EINEN Run durch:
    1. Balanciert das Training-Set neu (mit Seed!)
    2. Trainiert TabPFN
    3. Gibt Vorhersagen zurück
    """
    # A. Balancing (Das hast du oben manuell gemacht)
    # Hier nutzen wir den Seed, damit es reproduzierbar anders ist
    df_balanced = balance_data(df_train, n_samples=3000, seed=seed) 
    # (Hinweis: Du musst deine balance_data Funktion evtl. anpassen, dass sie einen 'seed' Parameter nimmt, 
    # sonst nimmst du einfach pandas sample mit random_state=seed)

    X_train, y_train = get_X_y(df_balanced)
    X_val_curr, y_val_curr = get_X_y(df_val) # X_val bleibt eigentlich gleich

    # B. Training
    classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=3)
    classifier.fit(X_train, y_train)
    
    # C. Prediction
    # Wir nehmen nur 1000 Samples für Speed, wie in deinem Entwurf
    X_val_sample = X_val_curr.iloc[:1000]
    y_val_real = y_val_curr[:1000]
    
    y_pred = classifier.predict(X_val_sample)
    
    return y_val_real, y_pred

def calculate_metrics(y_real, y_pred):
    """Berechnet die F1/Accuracy Scores."""
    return {
        'accuracy': accuracy_score(y_real, y_pred),
        'f1_macro': f1_score(y_real, y_pred, average='macro')
    }
# --- 2. EXPERIMENT ---

# Einmal Daten laden (bleibt konstant)
df_train, df_val, df_test = get_base_data()
results = []

print("🚀 Starte Robustness Loop...")

for i in range(30): # 30 Runs
    current_seed = 42 + i
    
    # Der eigentliche Job:
    try:
        y_real, y_pred = train_iteration(df_train, df_val, seed=current_seed)
        metrics = calculate_metrics(y_real, y_pred)
        
        # Tracking
        metrics['run_id'] = i
        metrics['seed'] = current_seed
        results.append(metrics)
        
        print(f"Run {i}: F1={metrics['f1_macro']:.3f}")
        
    except Exception as e:
        print(f"Fehler in Run {i}: {e}")

# Danach: Speichern und Plotten wie gehabt

# Vorschlag 2

In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np
import polars as pl
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.inspection import permutation_importance
from tabpfn import TabPFNClassifier

# Settings
N_LOOPS = 30 # Oder 1 für Test
RESULTS_DIR = f"../../reports/robustness_experiment/{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
os.makedirs(RESULTS_DIR, exist_ok=True)

# functions (siehe unten)
def get_balanced_train_set(df_train_pl, seed):
    """Erstellt ein balanciertes Training-Set durch Undersampling/Oversampling."""
    # Beispiel-Logik: Nimm von jeder Klasse gleich viele (z.B. Minimum der Klassen oder fester Wert)
    n_per_class = 333 # Oder dynamisch basierend auf der kleinsten Klasse
    
    # Hier nutzen wir Polars für Speed
    df_balanced = pl.concat([
        df_train_pl.filter(pl.col("target") == 0).sample(n=n_per_class, seed=seed, with_replacement=True), 
        df_train_pl.filter(pl.col("target") == 1).sample(n=n_per_class, seed=seed, with_replacement=True),
        df_train_pl.filter(pl.col("target") == 2).sample(n=n_per_class, seed=seed, with_replacement=False)
    ]).sample(fraction=1.0, shuffle=True, seed=seed)
    
    # Convert to Pandas/Numpy für TabPFN
    X = df_balanced.select(feature_cols).to_pandas()
    y = df_balanced.select("target").to_series().to_numpy()
    return X, y

def calculate_risk_stats(X_val, y_proba, feature_cols):
    """Berechnet durchschnittliche Risikoscores für Gruppen (für Plots später)."""
    df_temp = X_val.copy()
    df_temp['risk_score'] = y_proba[:, 0] # Annahme: Index 0 ist 'Early Event'
    
    stats = {}
    
    # 1. Kategorische Features
    cat_feats = [c for c in feature_cols if c not in ['bmi', 'anchor_age']]
    for cf in cat_feats:
        means = df_temp.groupby(cf, observed=True)['risk_score'].mean()
        for cat_val, val in means.items():
            safe_name = str(cat_val).replace(' ', '_').lower()[:20]
            stats[f'risk_{cf}_{safe_name}'] = val
            
    # 2. Age & BMI (vereinfacht)
    # Hier könntest du deine Bins-Logik reinpacken, wenn nötig
    stats['risk_bmi_mean'] = df_temp['bmi'].corr(df_temp['risk_score']) # Korrelation statt Bins spart Platz
    stats['risk_age_mean'] = df_temp['anchor_age'].corr(df_temp['risk_score'])
    
    return stats

def run_experiment_iteration(i, df_train_pl, X_val, y_val, feature_cols):
    """Führt EINEN kompletten Durchlauf durch (Ohne Plotting!)."""
    seed = 42 + i
    start_time = time.time()
    
    # 1. Data
    X_train_iter, y_train_iter = get_balanced_train_set(df_train_pl, seed)
    
    # 2. Fit
    clf = TabPFNClassifier(device='cpu', N_ensemble_configurations=3) # Ensemble niedrig -> schneller
    clf.fit(X_train_iter, y_train_iter)
    
    # 3. Predict (Der teure Teil)
    # Tipp: X_val muss hier nicht riesig sein. 1000 reichen für Robustness Check.
    y_pred = clf.predict(X_val)
    y_proba = clf.predict_proba(X_val)
    
    # 4. Metrics
    metrics = {
        'run_id': i,
        'seed': seed,
        'f1_macro': f1_score(y_val, y_pred, average='macro'),
        'accuracy': accuracy_score(y_val, y_pred),
        'duration': time.time() - start_time
    }
    
    # 5. Feature Importance (Teuer! Nur machen wenn nötig)
    # perm_result = permutation_importance(clf, X_val, y_val, n_repeats=2, ...)
    # metrics.update(...) vorerst weglassen für Speed oder Code einfügen
    
    # 6. Risk Stats
    metrics.update(calculate_risk_stats(X_val, y_proba, feature_cols))
    
    return metrics

# loop
results = []
print(f"🚀 Starte {N_LOOPS} Loops...")

# Wichtig: Polars DF für schnelles Sampling nutzen
df_train_pl = pl.from_pandas(pd.concat([X_train, pd.Series(y_train, name='target')], axis=1))

for i in range(N_LOOPS):
    # Nur Berechnen!
    res = run_experiment_iteration(i, df_train_pl, X_val_robust, y_val_robust, feature_cols)
    results.append(res)
    print(f"Run {i}: F1={res['f1_macro']:.3f} ({res['duration']:.1f}s)")

# Speichern
df_results = pd.DataFrame(results)
df_results.to_csv(os.path.join(RESULTS_DIR, "results.csv"), index=False)
print("Fertig gespeichert.")

# Robustness Experiment (Stabilitätstest)

Um zu beweisen, dass die Ergebnisse kein Zufall sind, führen wir das Training **30 Mal** mit unterschiedlichen Seeds durch.

**Ablauf:**
1. Ziehe in jedem Durchlauf ein **neues, balanciertes Training-Set** (Sampling Variation).
2. Trainiere ein frisches TabPFN Modell.
3. Evaluiere auf einem **fixen Validierungs-Set**.
4. Speichere Metriken (F1-Score, Accuracy) in einer CSV und visualisiere die Varianz.

Dies simuliert, wie robust das Modell gegenüber Veränderungen in den Trainingsdaten ist.

In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np
import polars as pl
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.inspection import permutation_importance
from tabpfn import TabPFNClassifier

# 1. Konfiguration
IS_TEST_RUN = True  # KIPPSCHALTER
TIMESTAMP = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if IS_TEST_RUN:
    N_LOOPS = 1
    BASE_DIR = "../../reports/robustness_experiment_test"
    print(f"⚠️ TEST-LAUF: Starte nur {N_LOOPS} Runde.")
else:
    N_LOOPS = 30
    BASE_DIR = "../../reports/robustness_experiment"
    print(f"🚀 PRODUKTIV-LAUF: Starte {N_LOOPS} Runden.")

RESULTS_DIR = os.path.join(BASE_DIR, TIMESTAMP)
os.makedirs(RESULTS_DIR, exist_ok=True)

X_val_robust = X_val.iloc[:1000].copy()
y_val_robust = y_val[:1000].copy()
results_list = []

print(f"🚀 Starte Experiment. Ergebnisse landen in: {RESULTS_DIR}")

# 2. Experiment Loop
for i in range(N_LOOPS):
    start_time = time.time()
    
    # Ordner für diesen Run
    run_dir = os.path.join(RESULTS_DIR, f"run_{i}")
    os.makedirs(run_dir, exist_ok=True)
    
    # A. Resampling
    current_seed = 42 + i
    n_per_class = 333
    
    df_iter_train = pl.concat([
        df_train_pl.filter(pl.col("target") == 0).sample(n=min(n_per_class, df_train_pl.filter(pl.col("target")==0).height), seed=current_seed),
        df_train_pl.filter(pl.col("target") == 1).sample(n=min(n_per_class, df_train_pl.filter(pl.col("target")==1).height), seed=current_seed),
        df_train_pl.filter(pl.col("target") == 2).sample(n=n_per_class, seed=current_seed)
    ]).sample(fraction=1.0, shuffle=True, seed=current_seed)
    
    X_train_iter = df_iter_train.select(feature_cols).to_pandas()
    y_train_iter = df_iter_train.select("target").to_series().to_numpy()
    
    # B. Training
    clf = TabPFNClassifier(device='cpu') 
    clf.fit(X_train_iter, y_train_iter)
    
    # C. Prediction
    y_pred = clf.predict(X_val_robust)
    y_proba = clf.predict_proba(X_val_robust)
    
    # Metriken
    f1_macro = f1_score(y_val_robust, y_pred, average='macro')
    acc = accuracy_score(y_val_robust, y_pred)
    prec_macro = precision_score(y_val_robust, y_pred, average='macro', zero_division=0)
    rec_macro = recall_score(y_val_robust, y_pred, average='macro', zero_division=0)
    f1_per_class = f1_score(y_val_robust, y_pred, average=None)

    # Feature Importance
    perm_result = permutation_importance(clf, X_val_robust, y_val_robust, n_repeats=2, random_state=42, n_jobs=1)
    
    duration = time.time() - start_time

    # --- RISK DIRECTION ANALYSE ---
    # Wir speichern durchschnittliche Risikoscores pro Gruppe in die CSV
    df_risk_check = X_val_robust.copy()
    df_risk_check['risk_score'] = y_proba[:, 0] # Klasse 0 = Early Event

    risk_stats = {}
    
    # 1. Kategorische Features (Gender, Insurance, etc.)
    cat_feats = [c for c in feature_cols if c not in ['bmi', 'anchor_age']]
    for cf in cat_feats:
        means = df_risk_check.groupby(cf, observed=True)['risk_score'].mean()
        for cat_val, val in means.items():
            # CSV-freundliche Spaltennamen
            safe_name = str(cat_val).replace(' ', '_').replace('/', '_').replace('-', '_').lower()[:20]
            risk_stats[f'risk_{cf}_{safe_name}'] = val

    # 2. Numerische Features (Bins)
    # BMI
    df_risk_check['bmi_grp'] = pd.cut(df_risk_check['bmi'], bins=[0, 18.5, 25, 30, 100], labels=['under', 'norm', 'over', 'obese'])
    for cat_val, val in df_risk_check.groupby('bmi_grp', observed=True)['risk_score'].mean().items():
        risk_stats[f'risk_bmi_{cat_val}'] = val
        
    # Age (Dekaden für Übersicht)
    df_risk_check['age_grp'] = pd.cut(df_risk_check['anchor_age'], bins=[0, 30, 50, 70, 90, 120], labels=['u30', '30_50', '50_70', '70_90', '90plus'])
    for cat_val, val in df_risk_check.groupby('age_grp', observed=True)['risk_score'].mean().items():
        risk_stats[f'risk_age_{cat_val}'] = val
    
    # Speichern
    result_dict = {
        'run_id': i, 'seed': current_seed, 'accuracy': acc, 'f1_macro': f1_macro,
        'precision_macro': prec_macro, 'recall_macro': rec_macro,
        'f1_class_0_early': f1_per_class[0], 'f1_class_1_late': f1_per_class[1], 'f1_class_2_healthy': f1_per_class[2],
        'duration_sec': duration
    }
    # Feature Importance hinzufügen
    for feat_name, importance_val in zip(feature_cols, perm_result.importances_mean):
        result_dict[f'imp_{feat_name}'] = importance_val
        
    # Risk-Stats hinzufügen
    result_dict.update(risk_stats)
    results_list.append(result_dict)

    # --- E. PLOTTING PRO RUN ---
    
    # 1. Confusion Matrix
    cm = confusion_matrix(y_val_robust, y_pred)
    cm_perc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    labels = ['Früh', 'Spät', 'Gesund']
    annot_text = [[f"{val}<br>({perc:.1%})" for val, perc in zip(r, rp)] for r, rp in zip(cm, cm_perc)]
    fig_cm = ff.create_annotated_heatmap(cm_perc, x=labels, y=labels, annotation_text=annot_text, colorscale='Reds')
    fig_cm.update_layout(title=f'Confusion Matrix (Run {i})', height=400, width=500)
    fig_cm.write_image(os.path.join(run_dir, "confusion_matrix.png"))

    # 1b. Sankey Diagram
    label_list = ["Tatsächlich: Früh", "Tatsächlich: Spät", "Tatsächlich: Gesund", 
                  "Vorhergesagt: Früh", "Vorhergesagt: Spät", "Vorhergesagt: Gesund"]
    source = [0, 0, 0, 1, 1, 1, 2, 2, 2] 
    target = [3, 4, 5, 3, 4, 5, 3, 4, 5] 
    value = cm.flatten()
    color_link = [
        'rgba(31, 119, 180, 0.4)', 'rgba(31, 119, 180, 0.2)', 'rgba(31, 119, 180, 0.1)', # Von Früh
        'rgba(255, 127, 14, 0.2)', 'rgba(255, 127, 14, 0.4)', 'rgba(255, 127, 14, 0.1)', # Von Spät
        'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.1)', 'rgba(44, 160, 44, 0.4)'    # Von Gesund
    ]
    fig_sankey = go.Figure(data=[go.Sankey(
        node = dict(pad = 15, thickness = 20, line = dict(color = "black", width = 0.5), label = label_list, color = "blue"),
        link = dict(source = source, target = target, value = value, color = color_link))])
    fig_sankey.update_layout(title_text=f"Patienten-Fluss (Run {i})", font_size=12, height=500)
    fig_sankey.write_image(os.path.join(run_dir, "sankey_flow.png"))
    
    # 2. Risk Plots Data Prep
    df_plot_run = X_val_robust.copy()
    df_plot_run['risk_score'] = y_proba[:, 0]
    
    # a) BMI
    bins_bmi = [0, 18.5, 25, 30, 100]
    labels_bmi = ['Untergewicht (<18.5)', 'Normal (18.5-25)', 'Übergewicht (25-30)', 'Adipositas (>30)']
    df_plot_run['bmi_group'] = pd.cut(df_plot_run['bmi'], bins=bins_bmi, labels=labels_bmi)
    df_bmi_agg = df_plot_run.groupby('bmi_group', observed=True)['risk_score'].mean().reset_index()
    
    fig_bmi = px.bar(df_bmi_agg, x='bmi_group', y='risk_score', text_auto='.1%', 
                     title=f'Risiko nach BMI-Gruppe (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'bmi_group': 'BMI Gruppe'},
                     template="plotly_white", height=400)
    fig_bmi.update_layout(yaxis_tickformat='.0%')
    fig_bmi.write_image(os.path.join(run_dir, "risk_bmi.png"))
    
    # b) Age
    bins_age = [0, 20, 30, 40, 50, 60, 70, 80, 90, 120]
    labels_age = ['<20', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90+']
    df_plot_run['age_group'] = pd.cut(df_plot_run['anchor_age'], bins=bins_age, labels=labels_age)
    df_plot_run['age_group'] = df_plot_run['age_group'].astype(str)
    
    df_age_agg = df_plot_run.groupby('age_group')['risk_score'].mean().reset_index()
    
    fig_age = px.bar(df_age_agg, x='age_group', y='risk_score', text_auto='.1%', 
                     title=f'Risiko-Entwicklung über das Alter (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'age_group': 'Altersgruppe'},
                     template="plotly_white", height=400)
    fig_age.update_layout(yaxis_tickformat='.0%')
    fig_age.write_image(os.path.join(run_dir, "risk_age.png"))

    # c) Insurance & Gender
    df_ins_agg = df_plot_run.groupby(['insurance', 'gender'])['risk_score'].mean().reset_index()
    fig_ins = px.bar(df_ins_agg, x='insurance', y='risk_score', color='gender', barmode='group',
                     text_auto='.1%', 
                     title=f'Risiko: Versicherung & Geschlecht (Run {i})', 
                     labels={'risk_score': 'Wahrscheinlichkeit', 'insurance': 'Versicherung'},
                     template="plotly_white", height=400)
    fig_ins.update_layout(yaxis_tickformat='.0%')
    fig_ins.write_image(os.path.join(run_dir, "risk_insurance_gender.png"))

    # 3. Feature Importance 
    imp_df_run = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': perm_result.importances_mean,
        'Std_Dev': perm_result.importances_std
    }).sort_values(by='Importance', ascending=True)

    fig_imp = px.bar(imp_df_run, x='Importance', y='Feature', orientation='h',
                     title=f'Feature Importance (Run {i})',
                     labels={'Importance': 'Wichtigkeit (Mean Decrease Accuracy)'}, error_x='Std_Dev',
                     template="plotly_white", color='Importance', color_continuous_scale='Reds')
    # NEU: Auch innerhalb der Runs Prozent-Formatierung für die Bilder
    fig_imp.update_layout(height=400, showlegend=False, xaxis_tickformat='.1%')
    fig_imp.write_image(os.path.join(run_dir, "feature_importance.png"))

    if i == 0 or (i+1) % 5 == 0:
        print(f"   Run {i+1}/{N_LOOPS}: F1={f1_macro:.3f} | Bilder in {run_dir}/")

# 3. Speichern
df_results = pd.DataFrame(results_list)
csv_path = os.path.join(RESULTS_DIR, "robustness_metrics_with_importance.csv")
df_results.to_csv(csv_path, index=False)
print(f"✅ Experiment abgeschlossen. Daten gespeichert in: {csv_path}")

⚠️ TEST-LAUF: Starte nur 1 Runde.
🚀 Starte Experiment. Ergebnisse landen in: ../../reports/robustness_experiment_test/2026-02-14_15-35-56


/Users/andrey/Repositories/fs-thesis/.venv/lib/python3.13/site-packages/tabpfn/validation.py:56: UserWarning:

Running on CPU with more than 200 samples may be slow.
Consider using a GPU or the tabpfn-client API: https://github.com/PriorLabs/tabpfn-client



   Run 1/1: F1=0.318 | Bilder in ../../reports/robustness_experiment_test/2026-02-14_15-35-56/run_0/
✅ Experiment abgeschlossen. Daten gespeichert in: ../../reports/robustness_experiment_test/2026-02-14_15-35-56/robustness_metrics_with_importance.csv


In [ ]:
# 4. Visualisierung der Robustness (Master-Thesis Style)
# Ziel: Durchschnittliche Performance zeigen + Stabilität (Fehlerbalken) beweisen

# Melt transformiert die Daten für Plotly (Wide -> Long Format)
df_melt = df_results.melt(
    id_vars=['run_id'], 
    value_vars=['f1_class_0_early', 'f1_class_1_late', 'f1_class_2_healthy'],
    var_name='Target Class', 
    value_name='F1 Score'
)

# Namen verschönern
df_melt['Target Class'] = df_melt['Target Class'].replace({
    'f1_class_0_early': 'Früher Ausbruch (<1J)',
    'f1_class_1_late': 'Später Ausbruch (1-3J)', 
    'f1_class_2_healthy': 'Kein Event / Gesund'
})

# Aggegierte Statistiken für Bar-Chart
df_stats = df_melt.groupby('Target Class')['F1 Score'].agg(['mean', 'std']).reset_index()

# Sortierung manuell festlegen (Logische Zeitreihe)
category_order = ['Früher Ausbruch (<1J)', 'Später Ausbruch (1-3J)', 'Kein Event / Gesund']

# 1. Bar Chart mit Error Bars (Klassisch wissenschaftlich)
fig = px.bar(
    df_stats, 
    x="Target Class", 
    y="mean", 
    error_y="std", # Zeigt die Standardabweichung als Antenne (Robustheit)
    title=f"Modell-Performance: Durchschnitt & Stabilität ({N_LOOPS} Runs)",
    text_auto='.1%', # Beschriftung direkt am Balken
    labels={'mean': 'Durchschnittlicher F1-Score'},
    color="Target Class", 
    # Pastell Farben wirken professioneller und weniger überladen
    color_discrete_sequence=px.colors.qualitative.Pastel,
    template="plotly_white",
    category_orders={"Target Class": category_order} # Erzwingt die logische zeitliche Reihenfolge
)

# Balken leicht transparent machen (0.8), damit sie nicht zu massiv wirken
fig.update_traces(marker_opacity=0.8, showlegend=False)

# 2. Scatter-Punkte darüber legen (Kontrastfarbe für bessere Sichtbarkeit)
# Zeigt jeden einzelnen Run als Punkt -> Maximale Transparenz der Ergebnisse
scatter_trace = px.strip(
    df_melt, 
    x="Target Class", 
    y="F1 Score", 
    category_orders={"Target Class": category_order}
).data[0]

# Styling der Punkte: Dunkles Kontrast-Blau mit weißem Rand
# Das sorgt für Lesbarkeit sowohl auf hellen als auch dunklen Hintergründen
scatter_trace.marker.color = '#34495e' # "Wet Asphalt" (Dunkles Grau-Blau)
scatter_trace.marker.size = 6
scatter_trace.marker.opacity = 0.8
scatter_trace.marker.line = dict(width=1, color='white') # Weißer Rand lässt Punkte "poppen"
scatter_trace.showlegend = False

fig.add_trace(scatter_trace)

# Achsen formatieren (Prozent statt 0.x)
fig.update_layout(
    yaxis_tickformat='.0%', 
    yaxis_title="F1 Score (Macro)", 
    xaxis_title=None, # X-Achsen Titel ist redundant wegen den Labels
    font=dict(size=14) # Schrift etwas größer für Thesis
)
fig.update_yaxes(range=[0, 1.1]) # Platz für Error Bars lassen

fig.write_image(os.path.join(RESULTS_DIR, "robustness_scientific_bar.png"))
fig.show()

In [ ]:
# 4b. Visualisierung der Feature Importance Stabilität
import pandas as pd
import plotly.express as px
import os

# Melt für Feature Wichtigkeit
feature_imp_cols = [c for c in df_results.columns if c.startswith("imp_")]
df_melt_imp = df_results.melt(
    id_vars=['run_id'], 
    value_vars=feature_imp_cols,
    var_name='Feature', 
    value_name='Importance Score'
)

# Prefix "imp_" für schönere Labels entfernen
df_melt_imp['Feature'] = df_melt_imp['Feature'].str.replace('imp_', '')

# Sortierung berechnen: Wir wollen das wichtigste Feature OBEN haben.
# Plotly Boxplots ordern die Y-Achse standardmäßig von unten nach oben.
# Wir sortieren hier nach Median aufsteigend (ascending=True).
# -> Das Feature mit dem kleinsten Median steht am Anfang der Liste (Index 0).
# -> Plotly zeichnet Index 0 unten.
# -> Das Feature mit dem größten Median steht am Ende der Liste.
# -> Plotly zeichnet das Ende oben.
# Falls es bei dir "falsch herum" ist, liegt es oft daran, dass Plotly 'Feature' noch als Faktor hat.
sorted_features = df_melt_imp.groupby('Feature')['Importance Score'].median().sort_values(ascending=False).index.tolist()

fig = px.box(
    df_melt_imp, 
    x="Importance Score", 
    y="Feature", 
    color="Feature", 
    # Wir erzwingen die Sortierung explizit über category_orders
    category_orders={"Feature": sorted_features}, 
    title=f"Feature Wichtigkeit über {N_LOOPS} Runs (Stabilitätstest)",
    points="all", 
    template="plotly_white",
    height=600,
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(showlegend=True)

# Punkte styling 
fig.update_traces(marker=dict(opacity=0.6, size=3)) # color='#34495e', 

# Achsen formatieren
fig.update_layout(
    xaxis_tickformat='.1%', 
    xaxis_title="Wichtigkeit (Mean Decrease Accuracy)",
    yaxis_title=None,
    font=dict(size=12)
)

# Speichern
fig.write_image(os.path.join(RESULTS_DIR, "robustness_feature_importance.png"))
fig.show()

In [ ]:
# 5. Sanity Check / Plausibilitäts-Prüfung
# Hinweis: RESULTS_DIR kommt jetzt aus dem Block oben mit Zeitstempel!

import pandas as pd
import os

print(f"Lade Prüfung von: {RESULTS_DIR}") # Nimmt den aktuellsten Ordner
try:
    csv_load_path = os.path.join(RESULTS_DIR, "robustness_metrics_with_importance.csv")
    check_df = pd.read_csv(csv_load_path)

    print(f"Anzahl durchgeführter Runs: {len(check_df)}")
    print(f"Genutzte Seeds: {check_df['seed'].unique()}")

    # Prüfung 1: Haben wir Variation?
    if len(check_df) > 1:
        std_dev = check_df['f1_macro'].std()
        if std_dev > 0:
            print(f"✅ Test bestanden: Ergebnisse variieren (Std-Dev: {std_dev:.4f}).")
        else:
            print("⚠️ Warnung: Keine Variation! Prüfe den Random Seed!")
    else:
        print("ℹ️ Hinweis: Nur 1 Run durchgeführt (Test-Modus).")

    # Prüfung 2: Sind die Ergebnisse 'zu perfekt'?
    if check_df['f1_macro'].max() >= 0.99:
        print("⚠️ Warnung: F1-Score von fast 1.0 (>=0.99) gefunden. Das ist verdächtig.")
    else:
        print(f"✅ Test bestanden: Realistische Scores (Max: {check_df['f1_macro'].max():.3f})")

    check_df.head()
    
except FileNotFoundError:
    print(f"❌ Fehler: Keine CSV gefunden in {RESULTS_DIR}. Hast du den Loop oben laufen lassen?")

Lade Prüfung von: ../../reports/robustness_experiment_test/2026-02-14_15-35-56
Anzahl durchgeführter Runs: 1
Genutzte Seeds: [42]
ℹ️ Hinweis: Nur 1 Run durchgeführt (Test-Modus).
✅ Test bestanden: Realistische Scores (Max: 0.318)


# 6. Test

In [ ]:
X_test_sample = X_test.iloc[:1000]
y_test_sample = y_test[:1000]

print("Starte Test...")
y_test_pred = classifier.predict(X_test_sample)
print("Fertig!")

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_sample, y_test_pred):.2%}")
print("\nClassification Report:")
print(classification_report(y_test_sample, y_test_pred, 
                            target_names=['Früh (<1J)', 'Spät (>1J)', 'Gesund']))